In [1]:
import numpy as np
import json
import re
import string

import tensorflow as tf
from tensorflow.keras import layers, models, losses

2024-07-19 19:52:13.678160: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-19 19:52:13.919425: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-19 19:52:13.919450: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-19 19:52:13.920923: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-19 19:52:14.035981: I tensorflow/core/platform/cpu_feature_g

In [3]:
# 데이터셋은 http://oreil.ly/laNUt 에서 직접 다운로드 받았습니다.
with open('./full_format_recipes.json') as json_data:
    recipe_data = json.load(json_data)
    
filtered_data = [
    'Recipe for ' + x['title'] + " | " + ''.join(x['directions']) for x in recipe_data
    if 'title' in x and x['title'] is not None and 'directions' in x and x['directions'] is not None
]

In [5]:
n_recipes = len(filtered_data)
print('{}개 레시피 로드'.format(n_recipes))

example = filtered_data[19]
print(example)

20111개 레시피 로드
Recipe for Tuna, Asparagus, and New Potato Salad with Chive Vinaigrette and Fried Capers  | Puree first 5 ingredients in blender until smooth. With machine running, gradually add vegetable oil, then olive oil. Season to taste with salt and pepper. DO AHEAD: Can be made 1 day ahead. Cover and chill.Cook asparagus in large skillet of boiling salted water until just tender, 4 to 5 minutes. Transfer asparagus to 13x9x2-inch pan of ice water to cool. Drain asparagus and pat dry. DO AHEAD: Can be made 8 hours ahead. Wrap in paper towels, then plastic, and chill.Place potatoes in large saucepan. Add enough water to cover potatoes by 1 inch. Sprinkle with salt. Bring to boil and cook until potatoes are tender, 10 to 15 minutes, depending on size of potatoes. Drain; let cool 5 minutes. Place in medium bowl. Add 1/4 cup vinaigrette; toss to coat. Season to taste with salt and pepper.Heat olive oil in small skillet over medium-high heat. Add capers and fry until capers are crisp and

### 토큰화(tokenization)

토큰화는 텍스트를 단어나 문자와 같은 개별 단위로 나누는 작업.  
텍스트 토큰화는 다양한 방법이 존재하고 방법에 따라 이후 모델의 출력에 많은 영향을 미칩니다.

In [6]:
def pad_punctuation(s):
    s = re.sub(f'([{string.punctuation}])', r' \1', s)
    s = re.sub(' +', ' ', s)
    return s

text_data = [pad_punctuation(x) for x in filtered_data]
text_ds = tf.data.Dataset.from_tensor_slices(text_data).batch(32).shuffle(1000)

vectorize_layer = layers.TextVectorization(
    standardize = 'lower',
    max_tokens = 10000,
    output_mode = 'int',
    output_sequence_length = 200 + 1,
)

vectorize_layer.adapt(text_ds)
vocab = vectorize_layer.get_vocabulary()

2024-07-19 20:10:33.163827: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-19 20:10:33.270546: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-19 20:10:33.270578: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-19 20:10:33.272059: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-19 20:10:33.272083: I tensorflow/compile

In [7]:
for i, word in enumerate(vocab[:10]):
    print('{}: {}'.format(i, word))

0: 
1: [UNK]
2: ,
3: .
4: and
5: to
6: in
7: the
8: with
9: a


In [8]:
example_data = text_data[9]
example_tokenized = vectorize_layer(example_data)
print(example_tokenized.numpy())

2024-07-19 20:13:54.905456: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


[  24   14  571    1    8  306  340  186    4 1103  508   25  342  224
  233  256    5  615   11  129   20  357    3  342   40  256    4  694
    4   66    8  166    4   74    6    9   60   72    2  110    2   56
   12  333    2   85  674   18   33    6    9   28   19    4   63  535
   11  158    3  350  166   95    9  373  679  303   22   19    8  674
    4   38    5  988    3   61    8   21    4   30    3  107   19    6
  173  176 1324    4   55    5  134  105    2   44  721  571    8  278
  233    4  253  285 1039    3  114  674   26   68    4  102   10  107
    2   50  198   11  168  433  103  489    2  306    2  188    2   11
   67  138   21    2    4   11   67  138   30    6    9   27   19    3
   38    6  353    2 3758    2    4  145 1519  508    8 1347    2   31
    2   11   53  138   30    2    4   11   67  138   21    6    9  286
  184    5    9  418  608 3298  508    2   41  340  186    2   18  571
    3    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [10]:
# 훈련 세트 만들기
def prepare_inputs(text):
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y


train_ds = text_ds.map(prepare_inputs)

### LSTM 만들기

모델의 입력 = 정수 토큰의 시퀀스,  
모델의 출력 = 10,000개 단어의 어휘 사전에서 시퀀스 다음에 나올 단어의 확률

In [11]:
# 파라미터
VOCAB_SIZE = 10000
MAX_LEN = 200
EMBEDDING_SIZE = 100
N_UNITS = 128
VALIDATION_SPLIT = 0.2
SEED = 42
LOAD_MODEL = False
BATCH_SIZE = 32
EPOCHS = 25

In [12]:
inputs = layers.Input(shape=(None,), dtype='int32')
x = layers.Embedding(VOCAB_SIZE, EMBEDDING_SIZE)(inputs)
x = layers.LSTM(N_UNITS, return_sequences=True)(x)
outputs = layers.Dense(VOCAB_SIZE, activation='softmax')(x)
lstm = models.Model(inputs, outputs)
lstm.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         1000000   
                                                                 
 lstm (LSTM)                 (None, None, 128)         117248    
                                                                 
 dense (Dense)               (None, None, 10000)       1290000   
                                                                 
Total params: 2407248 (9.18 MB)
Trainable params: 2407248 (9.18 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
